## Using Context Items

Context is a global memory space available to rules to store and retrieve information about items. This is a key concept to know as this is a way for rules to communicate with each other during processing.

Items are a generic term that represents a collection of knowledge about a single thing. A common way to use items is to store information about real-world entities, such as a person, place, or other object. Items can have attributes, and those attributes can be other nested values so that you can store hierarchical information about those items.

If you are familiar with Blackboard systems, the Context is the area where all agents can write to and read from.

During processing, rules can work together to contribute and add knowledge to items. For example, in a natural language processing system, you may have rules which detect noun phrases. These rules can work together to continually append information into a Context item, similiar to how we build up an image or understanding in our minds when we hear words describing an object.

In [1]:
import os, sys
sys.path.insert(1, os.path.abspath('..\\..'))
from thoughts.rules_engine import RulesEngine
import pprint

engine = RulesEngine()
pprint.pprint(engine.context.items)

{}


## Naming Items

Each item has a name, starting with a "$" character. You can use this name to refer to the item, as well as attributes on the item by using "dot" syntax. For example, you can have an item representing a person, track information about a person, and refer to that information with $person.phone_number.

The code below adds an item manually to the Context. This is useful if you want to preload the Context with some information from a database or file. Upcoming sections will show you how to do this within the rules themselves to read and write information during rule processing.

In [2]:
engine.context.items["$tom"] = {"birthdate": "01/01/2005"}

pprint.pprint(engine.context.items)

{'$tom': {'birthdate': '01/01/2005'}}


## Storing Simple Values

The #store command will store an item into the Context, similiar to how variables are stored in memory. Note that we use the term "item" when referring to an item in the Context.

Note that the local variable's name starts with "?" and the Context item's name starts with "$". This is to help mentally separate temporary variables from more the persistant items in the Context.

In [3]:
rule = {
    "#when": "animal",
    "#then": [{ "#store": "dog", "#into": "$animal" }, { "#output": "$animal" }]
}

engine.add_rule(rule)
engine.run_assert("animal")
pprint.pprint(engine.context.items)

dog
{'$animal': 'dog', '$tom': {'birthdate': '01/01/2005'}}


## Storing Values Passed In

You can also store values that are passed from the #when clause to the #then clauses.

In this example, the ?animal variable is set in the #when clause and then used in the #then clause to store the value into the $animal Context item.

Notice here too that the previous item's value is overrwritten with the new value.

In [4]:
rule = {
    "#when": "the animal is a ?animal",
    "#then": [{ "#store": "?animal", "#into": "$animal" }, { "#output": "$animal" }]
}

engine.add_rule(rule)
engine.run_assert("the animal is a cat")
pprint.pprint(engine.context.items)

cat
{'$animal': 'cat', '$tom': {'birthdate': '01/01/2005'}}


## Handling Unknown Values

If the local variable is not set in the #when clause, then the variable's name will be used.

Likewise, if an item is not found in the Context, then the item's name will be used.

In [5]:
rules = [{
    "#when": "sport",
    "#then": [{ "#output": "?sport" }]
  },
{
    "#when": "tree",
    "#then": [{ "#output": "$tree" }]
}]

engine.context.add_ruleset(rules, "unknown-values")
engine.run_assert("sport")
engine.run_assert("tree")

?sport
$tree


## Appending Values

By default, the #store command will replace the previous value with the new one. You can append the item to the current value by using the #append option, which creates a list.

In [6]:
rule = {
    "#when": "add ?car to the cars list",
    "#then": [{ "#store": "jeep", "#into": "$cars" }, 
             { "#store": "?car", "#append": "$cars" },
             { "#output": "cars are $cars" }]
}

engine.add_rule(rule)
engine.run_assert("add honda to the cars list")
pprint.pprint(engine.context.items)

cars are ['jeep', 'honda']
{'$animal': 'cat',
 '$cars': ['jeep', 'honda'],
 '$tom': {'birthdate': '01/01/2005'}}


## Appending Values to the Beginning

Or, if you need to place the item at the beginning of the sequence, you can do that using the #push option.

In [7]:
rule = {
    "#when": "add ?food to the foods list",
    "#then": [{ "#store": "apple", "#into": "$foods" }, 
             { "#store": "?food", "#push": "$foods" },
             { "#output": "foods are $foods" }]
}

engine.add_rule(rule)
engine.run_assert("add orange to the foods list")
pprint.pprint(engine.context.items)

foods are ['orange', 'apple']
{'$animal': 'cat',
 '$cars': ['jeep', 'honda'],
 '$foods': ['orange', 'apple'],
 '$tom': {'birthdate': '01/01/2005'}}


In [ ]:
rules = [

  {"#item": "$dog1", "breed": "havanese"},

  {
    "#when": "doggo",
    "#then": "$dog1.breed"
  },

  {
    "#when": "get dog breed",
    "#then": [
      { "#assert": "doggo", "into": "?doginfo" },
      { "#output": "?doginfo"}
    ]
  }

]